In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("/Users/henryfetzer/Documents/ACME/Vol3FallProj/data/Life_Expectancy_Data.csv")

In [3]:
print(df.shape)
df = df.dropna()
df = df.rename(columns={'Life expectancy ': 'Life expectancy', 'under-five deaths ':'under-five deaths',
                        'Measles ':'Measles'})
df.drop(columns=['Country', 'Status'], inplace=True)
print(df.shape)


(2938, 22)
(1649, 20)


In [4]:
# Create target column
df['life_over_70'] = (df['Life expectancy'] >= 70).astype(int)

In [5]:
# Train test split
data = df.drop(columns=['life_over_70', 'Life expectancy', 'Adult Mortality'])
target = df['life_over_70']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

print(X_train.shape)

(1319, 18)


In [11]:
xgb_classifier = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

param_grid = {"alpha": [0.1, 0.2, 1],
              "gamma": [0.1,0.08, 0.2],
              "lambda": [0.5, 1],
              "eta": [0.4,0.6,0.7]}
gs = GridSearchCV(xgb_classifier, param_grid, scoring='f1_macro', n_jobs=-1, verbose=3, cv=2)

# run the search
gs.fit(X_train, y_train)


# print best parameters and resulting score
print(gs.best_params_, gs.best_score_, sep='\n')

Fitting 2 folds for each of 54 candidates, totalling 108 fits
[CV 2/2] END alpha=0.1, eta=0.4, gamma=0.1, lambda=1;, score=0.954 total time=   0.2s
[CV 1/2] END alpha=0.1, eta=0.4, gamma=0.1, lambda=0.5;, score=0.946 total time=   0.2s
[CV 2/2] END alpha=0.1, eta=0.4, gamma=0.1, lambda=0.5;, score=0.958 total time=   0.2s
[CV 1/2] END alpha=0.1, eta=0.4, gamma=0.1, lambda=1;, score=0.948 total time=   0.2s
[CV 1/2] END alpha=0.1, eta=0.4, gamma=0.08, lambda=1;, score=0.946 total time=   0.2s
[CV 2/2] END alpha=0.1, eta=0.4, gamma=0.08, lambda=0.5;, score=0.957 total time=   0.2s
[CV 1/2] END alpha=0.1, eta=0.4, gamma=0.08, lambda=0.5;, score=0.949 total time=   0.2s
[CV 2/2] END alpha=0.1, eta=0.4, gamma=0.08, lambda=1;, score=0.961 total time=   0.2s
[CV 1/2] END alpha=0.1, eta=0.4, gamma=0.2, lambda=0.5;, score=0.951 total time=   0.2s
[CV 1/2] END alpha=0.1, eta=0.4, gamma=0.2, lambda=1;, score=0.946 total time=   0.1s
[CV 2/2] END alpha=0.1, eta=0.4, gamma=0.2, lambda=0.5;, score=0

In [13]:
# zip features importance and names together
feature_names = data.columns
features_and_importance = sorted(zip(gs.best_estimator_.feature_importances_, feature_names), 
                                 reverse=True)

for importance, name in features_and_importance:
    print(f"{name}: {importance}")

Income composition of resources: 0.34938961267471313
 thinness 5-9 years: 0.09346988797187805
GDP: 0.08584621548652649
 HIV/AIDS: 0.08139563351869583
 thinness  1-19 years: 0.07878314703702927
Total expenditure: 0.05293281376361847
infant deaths: 0.035944558680057526
Schooling: 0.03261682018637657
Alcohol: 0.0294638704508543
Polio: 0.02524624578654766
percentage expenditure: 0.018992658704519272
under-five deaths: 0.01840507797896862
Diphtheria : 0.018251385539770126
 BMI : 0.017909187823534012
Year: 0.017763130366802216
Measles: 0.015683604404330254
Population: 0.015268041752278805
Hepatitis B: 0.012638079933822155


The result is close, but the accuracy is a little worse. I would probably have to spend more time tuning hyperparameters.